In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import t
plt.rcParams.update({'figure.max_open_warning': 0})
import matplotlib
import glob
import os

from scipy.stats import trim_mean
from statsmodels import robust
from matplotlib import font_manager, rc
%matplotlib inline
from matplotlib import cm 
import gc  # garbage collection
from sklearn.cluster import KMeans  # k-means 군집분석
from sklearn.metrics import silhouette_score  # 군집분석 평가를 위한 silhouette 값 계산

pd.set_option('max_columns', 20, 'max_rows', 20)

%matplotlib inline

# 한글 폰트 설정|
import platform
your_os = platform.system()
if your_os == 'Linux':
    rc('font', family='NanumGothic')
elif your_os == 'Windows':
    ttf = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=ttf).get_name()
    rc('font', family=font_name)
elif your_os == 'Darwin':
    rc('font', family='AppleGothic')
rc('axes', unicode_minus=False)

import datetime
import time

In [2]:
new_feature = pd.read_csv('../data/0930피쳐.csv', encoding='cp949')

In [3]:
dogjang = pd.read_csv('../data/날짜별_개장여부.csv', encoding='cp949')

In [4]:
dogjang

,날짜,개장
0,2016-01-01,0
1,2016-01-02,0
2,2016-01-03,0
3,2016-01-04,1
4,2016-01-05,1
...,...,...
2187,2021-12-27,1
2188,2021-12-28,1
2189,2021-12-29,1
2190,2021-12-30,1


In [5]:
dogjang['날짜'] = dogjang['날짜'].apply(lambda x: datetime.datetime.strptime(str(x),'%Y-%m-%d'))

In [7]:
new_feature

,계좌번호,기준일자,종목코드,주당액면가,종목보유잔고수량,종목보유잔고수량_shift,전체종목평가금액,전체종목평가금액_shift,일_매수매도_수량,일_매수매도_총금액,일_매수매도_주당금액,계좌별_종목별_일_매수_수량_평균,계좌별_종목별_일_매도_수량_평균,계좌별_종목별_총보유기간,계좌별_종목별_마지막처음_차액
0,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-03-21,A002270,5000.0,1.0,0.0,667000.0,0.0,1.0,667000.0,667000.0,1.666667,5.0,516.0,-313500.0
1,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-03-28,A002270,5000.0,2.0,1.0,1270000.0,667000.0,1.0,603000.0,603000.0,1.666667,5.0,516.0,-313500.0
2,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-05-07,A002270,5000.0,3.0,2.0,1815000.0,1270000.0,1.0,545000.0,545000.0,1.666667,5.0,516.0,-313500.0
3,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-07-16,A002270,5000.0,4.0,3.0,2104000.0,1815000.0,1.0,289000.0,289000.0,1.666667,5.0,516.0,-313500.0
4,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2020-01-29,A002270,5000.0,8.0,4.0,2964000.0,2104000.0,4.0,860000.0,215000.0,1.666667,5.0,516.0,-313500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2573804,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-10-13,A293490,100.0,4.0,3.0,196600.0,198900.0,1.0,-2300.0,-2300.0,1.250000,5.0,91.0,-24700.0
2573805,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-11-24,A293490,100.0,5.0,4.0,245500.0,196600.0,1.0,48900.0,48900.0,1.250000,5.0,91.0,-24700.0
2573806,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-12-14,A293490,100.0,0.0,5.0,0.0,245500.0,-5.0,-245500.0,49100.0,1.250000,5.0,91.0,-24700.0
2573807,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-09-10,A298040,5000.0,15.0,0.0,1072500.0,0.0,15.0,1072500.0,71500.0,15.000000,15.0,5.0,0.0


In [8]:
new_feature['기준일자'] = new_feature.기준일자.apply(lambda x: datetime.datetime.strptime(str(x),'%Y-%m-%d'))

In [9]:
new_feature

,계좌번호,기준일자,종목코드,주당액면가,종목보유잔고수량,종목보유잔고수량_shift,전체종목평가금액,전체종목평가금액_shift,일_매수매도_수량,일_매수매도_총금액,일_매수매도_주당금액,계좌별_종목별_일_매수_수량_평균,계좌별_종목별_일_매도_수량_평균,계좌별_종목별_총보유기간,계좌별_종목별_마지막처음_차액
0,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-03-21,A002270,5000.0,1.0,0.0,667000.0,0.0,1.0,667000.0,667000.0,1.666667,5.0,516.0,-313500.0
1,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-03-28,A002270,5000.0,2.0,1.0,1270000.0,667000.0,1.0,603000.0,603000.0,1.666667,5.0,516.0,-313500.0
2,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-05-07,A002270,5000.0,3.0,2.0,1815000.0,1270000.0,1.0,545000.0,545000.0,1.666667,5.0,516.0,-313500.0
3,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-07-16,A002270,5000.0,4.0,3.0,2104000.0,1815000.0,1.0,289000.0,289000.0,1.666667,5.0,516.0,-313500.0
4,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2020-01-29,A002270,5000.0,8.0,4.0,2964000.0,2104000.0,4.0,860000.0,215000.0,1.666667,5.0,516.0,-313500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2573804,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-10-13,A293490,100.0,4.0,3.0,196600.0,198900.0,1.0,-2300.0,-2300.0,1.250000,5.0,91.0,-24700.0
2573805,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-11-24,A293490,100.0,5.0,4.0,245500.0,196600.0,1.0,48900.0,48900.0,1.250000,5.0,91.0,-24700.0
2573806,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-12-14,A293490,100.0,0.0,5.0,0.0,245500.0,-5.0,-245500.0,49100.0,1.250000,5.0,91.0,-24700.0
2573807,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-09-10,A298040,5000.0,15.0,0.0,1072500.0,0.0,15.0,1072500.0,71500.0,15.000000,15.0,5.0,0.0


In [12]:
ccc = []
cnt = 1
start = time.time()
for i in list(new_feature.계좌번호.unique()):
    sam_df = new_feature[new_feature.계좌번호 == i]
    for j in list(sam_df.종목코드.unique()):
        sam_sam_df = sam_df[sam_df.종목코드 == j]
        total_length = len(sam_sam_df)
        day_first = sam_sam_df.iloc[0,:].기준일자
        day_last = sam_sam_df.iloc[-1,:].기준일자
        x = [dogjang.query("(@day_first <= 날짜) and (날짜 <= @day_last)").개장.sum()]
        ccc.extend(x*total_length)
        if cnt % 10000 == 0:
            print(cnt,'번째 완료')
            print('소요시간', round((time.time() - start))/60,'분')
            print('-------------------------')
        cnt += 1

10000 번째 완료
소요시간 1.0166666666666666 분
-------------------------
20000 번째 완료
소요시간 2.05 분
-------------------------
30000 번째 완료
소요시간 2.9166666666666665 분
-------------------------
40000 번째 완료
소요시간 3.8833333333333333 분
-------------------------
50000 번째 완료
소요시간 4.75 분
-------------------------
60000 번째 완료
소요시간 5.683333333333334 분
-------------------------
70000 번째 완료
소요시간 6.633333333333334 분
-------------------------
80000 번째 완료
소요시간 7.583333333333333 분
-------------------------
90000 번째 완료
소요시간 8.633333333333333 분
-------------------------
100000 번째 완료
소요시간 9.55 분
-------------------------
110000 번째 완료
소요시간 10.416666666666666 분
-------------------------
120000 번째 완료
소요시간 11.316666666666666 분
-------------------------
130000 번째 완료
소요시간 12.2 분
-------------------------
140000 번째 완료
소요시간 13.216666666666667 분
-------------------------
150000 번째 완료
소요시간 14.266666666666667 분
-------------------------
160000 번째 완료
소요시간 15.166666666666666 분
-------------------------
170000 번째 완료
소요시간 16.13333333

In [13]:
new_feature['계좌별_종목별_총보유기간_real'] = ccc

In [14]:
new_feature = new_feature.drop(columns=['계좌별_종목별_총보유기간'])

In [15]:
new_feature.to_csv('0930피쳐수정.csv', encoding='cp949', index=False)